<a href="https://colab.research.google.com/github/lhlanh/lhlanh/blob/main/chatbot%20sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import re

# Đọc dữ liệu sản phẩm
df = pd.read_csv("clothes_chatbot_sanpham.csv")
df["Tên sản phẩm"] = df["Tên sản phẩm"].str.lower()
df["Giới tính"] = df["Giới tính"].str.lower()

# Ghi nhớ trạng thái trò chuyện
user_gender = None
last_product_type = None  # ghi nhớ sản phẩm người dùng hỏi gần nhất

# Xác định loại sản phẩm
def get_product_type(query):
    áo_keywords = ["áo", "hoodie", "croptop", "khoác", "polo", "sơ mi"]
    quần_keywords = ["quần", "jean", "tây", "short"]
    váy_keywords = ["váy", "đầm"]

    for word in váy_keywords:
        if word in query:
            return "váy"
    for word in quần_keywords:
        if word in query:
            return "quần"
    for word in áo_keywords:
        if word in query:
            return "áo"
    return None

# Tư vấn size dựa vào giới tính, chiều cao, cân nặng, loại sản phẩm
def suggest_size(query, gender, product_type):
    height_match = re.search(r"(\d{1,2}(\.\d{1,2})?)m", query)  # tìm chiều cao (m)
    weight_match = re.search(r"(\d{2,3})kg", query)  # tìm cân nặng (kg)

    if not gender:
        return "Bạn vui lòng cho biết giới tính (nam hoặc nữ) để tôi tư vấn size chính xác hơn."
    if not product_type:
        return "Bạn vui lòng nói rõ bạn muốn tư vấn size cho sản phẩm gì (ví dụ: áo, quần, váy...)"

    if height_match and weight_match:
        height = float(height_match.group(1)) * 100  # đổi chiều cao từ m sang cm
        weight = int(weight_match.group(1))

        if gender == "nam":
            if product_type == "áo":
                if height < 165 and weight <= 58:
                    return "Bạn là nam, nên mặc áo size S."
                elif height < 170 and weight <= 66:
                    return "Bạn là nam, nên mặc áo size M."
                elif height < 175 and weight <= 74:
                    return "Bạn là nam, nên mặc áo size L."
                elif height < 180 and weight <= 82:
                    return "Bạn là nam, nên mặc áo size XL."
                else:
                    return "Bạn là nam, nên mặc áo size XXL."
            elif product_type == "quần":
                if height < 165 and weight <= 58:
                    return "Bạn là nam, nên mặc quần size 28–29."
                elif height < 170 and weight <= 66:
                    return "Bạn là nam, nên mặc quần size 30–31."
                elif height < 175 and weight <= 74:
                    return "Bạn là nam, nên mặc quần size 32–33."
                else:
                    return "Bạn là nam, nên mặc quần size 34+."
        elif gender == "nữ":
            if product_type == "áo":
                if height < 158 and weight <= 47:
                    return "Bạn là nữ, nên mặc áo size S."
                elif height < 164 and weight <= 54:
                    return "Bạn là nữ, nên mặc áo size M."
                elif height < 170 and weight <= 62:
                    return "Bạn là nữ, nên mặc áo size L."
                elif height < 175 and weight <= 70:
                    return "Bạn là nữ, nên mặc áo size XL."
                else:
                    return "Bạn là nữ, nên mặc áo size XXL."
            elif product_type == "quần" or product_type == "váy":
                if height < 158 and weight <= 47:
                    return "Bạn là nữ, nên mặc quần/váy size 25–26."
                elif height < 164 and weight <= 54:
                    return "Bạn là nữ, nên mặc quần/váy size 27–28."
                elif height < 170 and weight <= 62:
                    return "Bạn là nữ, nên mặc quần/váy size 29–30."
                else:
                    return "Bạn là nữ, nên mặc quần/váy size 31+."
    return "Xin lỗi, tôi chưa đủ thông tin để tư vấn size."

def chatbot(query):
    global user_gender, last_product_type
    query = query.lower()

    # Ghi nhớ giới tính
    if "tôi là nam" in query or "nam" == query.strip():
        user_gender = "nam"
        return "Cảm ơn bạn. Bạn có thể hỏi về sản phẩm hoặc tư vấn size nhé!"
    elif "tôi là nữ" in query or "nữ" == query.strip():
        user_gender = "nữ"
        return "Cảm ơn bạn. Bạn có thể hỏi về sản phẩm hoặc tư vấn size nhé!"

    #tư vấn size (có chiều cao/cân nặng)
    if "m" in query and "kg" in query:
        product_type = get_product_type(query) or last_product_type
        return suggest_size(query, user_gender, product_type) or "Xin lỗi, tôi chưa đủ thông tin để tư vấn size."

    #sản phẩm
    matched_products = []
    found_type = get_product_type(query)
    if found_type:
        last_product_type = found_type

    for _, row in df.iterrows():
        if user_gender and user_gender not in row["Giới tính"] and row["Giới tính"] != "unisex":
            continue
        if row["Tên sản phẩm"] in query or any(word in query for word in row["Tên sản phẩm"].split()):
            matched_products.append(f'{row["Tên sản phẩm"].title()}: {row["Mô tả"]}. Giá: {row["Giá tiền"]}')

    if matched_products:
        return "\n\n".join(matched_products)

    return "Xin lỗi, tôi chưa có thông tin về sản phẩm đó. Bạn có thể hỏi loại khác không?"

print("🤖 Xin chào! Tôi là chatbot bán quần áo.")
print("Bạn có thể hỏi về sản phẩm hoặc tư vấn size (ví dụ: 'Tôi là nữ, cao 1m60, nặng 50kg, muốn mua váy')")
print("(Gõ 'thoát' để kết thúc)")

while True:
    user_input = input("Bạn: ")
    if user_input.lower() in ["thoát", "exit", "quit"]:
        print("🤖 Tạm biệt! Hẹn gặp lại.")
        break
    response = chatbot(user_input)
    print("Chatbot:", response)q

🤖 Xin chào! Tôi là chatbot bán quần áo.
Bạn có thể hỏi về sản phẩm hoặc tư vấn size (ví dụ: 'Tôi là nữ, cao 1m60, nặng 50kg, muốn mua váy')
(Gõ 'thoát' để kết thúc)
Bạn: nam
Chatbot: Cảm ơn bạn. Bạn có thể hỏi về sản phẩm hoặc tư vấn size nhé!
Bạn: quần
Chatbot: Quần Tây Nam: Quần tây nam lịch sự, vải kháng nhăn. Giá: 320.000 VNĐ

Quần Short Nam: Quần short kaki nam, thoải mái, có túi. Giá: 200.000 VNĐ
Bạn: cao 1m72, nặng 60kg
Chatbot: Bạn là nam, nên mặc quần size 30–31.
Bạn: nam
Chatbot: Cảm ơn bạn. Bạn có thể hỏi về sản phẩm hoặc tư vấn size nhé!
Bạn: áo
Chatbot: Áo Thun Nam: Áo thun cotton 100%, co giãn, thoáng mát. Giá: 150.000 VNĐ

Áo Hoodie: Hoodie nam nữ, vải nỉ ấm, nhiều màu sắc. Giá: 350.000 VNĐ

Áo Polo Nam: Áo polo nam lịch sự, thoáng khí. Giá: 250.000 VNĐ
Bạn: cao 1m72, nặng 60kg
Chatbot: Bạn là nam, nên mặc áo size M.
Bạn: nữ
Chatbot: Cảm ơn bạn. Bạn có thể hỏi về sản phẩm hoặc tư vấn size nhé!
Bạn: uần
Chatbot: Xin lỗi, tôi chưa có thông tin về sản phẩm đó. Bạn có thể hỏ